In [1]:
!pip install comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 762.0/762.0 kB 14.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 80.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: python-box
    Found existing installation: python-box 7.3.2
    Uninstalling python-box-7.3.2:
      Successfully uninstalled python-box-7.3.2


In [2]:
!pip install coqui-tts -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 65.9 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 84.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 90.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 40.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 99.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.5/857.5 kB 42.

In [3]:
%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


In [4]:
from IPython.display import Audio, display
from trainer import Trainer, TrainerArgs
from TTS.api import TTS
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig #, XttsAudioConfig
from TTS.tts.models.xtts import XttsAudioConfig
from TTS.utils.manage import ModelManager
# from TTS.utils.distribute import init_distributed

from tqdm.notebook import tqdm
import torch
import librosa
import sys
import os
import comet_ml


2025-09-04 12:55:00.460985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756990500.658717      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756990500.717942      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
comet_ml.login(project_name="gothic-xtts-finetune")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


Comet API key:  ········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [6]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="metadata.txt",
    language="ru",
    path="/kaggle/input/ljspeech-gothic/15"
)

# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, eval_split_size=0.02)

In [7]:
train_samples[:5]

[{'text': 'Вот, 10 кусков.\n',
  'audio_file': '/kaggle/input/ljspeech-gothic/15/wavs/INFO_ERPRESSER_CHOICE_ZAHLEN_15_01.wav',
  'speaker_name': 'ljspeech',
  'root_path': '/kaggle/input/ljspeech-gothic/15',
  'language': 'ru',
  'audio_unique_name': '#wavs/INFO_ERPRESSER_CHOICE_ZAHLEN_15_01'},
 {'text': 'Я могу помочь тебе продавать болотник - но не задаром, конечно.\n',
  'audio_file': '/kaggle/input/ljspeech-gothic/15/wavs/DIA_BAALISIDRO_GIMMEKRAUT_15_00.wav',
  'speaker_name': 'ljspeech',
  'root_path': '/kaggle/input/ljspeech-gothic/15',
  'language': 'ru',
  'audio_unique_name': '#wavs/DIA_BAALISIDRO_GIMMEKRAUT_15_00'},
 {'text': 'Этот демон - это и есть Спящий?\n',
  'audio_file': '/kaggle/input/ljspeech-gothic/15/wavs/INFO_XARDAS_BARRIER_15_06.wav',
  'speaker_name': 'ljspeech',
  'root_path': '/kaggle/input/ljspeech-gothic/15',
  'language': 'ru',
  'audio_unique_name': '#wavs/INFO_XARDAS_BARRIER_15_06'},
 {'text': 'Дай мне эти магические свитки.\n',
  'audio_file': '/kaggle/i

In [ ]:
min_audio_length = 99999999999 
max_audio_length = 0
for sample in tqdm(train_samples + eval_samples):
    array, _ = librosa.load(sample["audio_file"])
    if len(array) > max_audio_length:
        max_audio_length = len(array)
    if len(array) < min_audio_length:
        min_audio_length = len(array)

In [ ]:
min_audio_length, max_audio_length

In [8]:
OUT_PATH = '/kaggle/working/run/'
os.makedirs(OUT_PATH, exist_ok=True)

In [9]:
# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)

# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=False)

# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=False
    )

 > Downloading DVAE files!


  0%|          | 0.00/1.07k [00:00<?, ?iB/s]
100%|██████████| 1.07k/1.07k [00:00<00:00, 5.07kiB/s]

  5%|▍         | 9.63M/211M [00:00<00:02, 96.3MiB/s]
  9%|▉         | 19.3M/211M [00:00<00:01, 96.6MiB/s]
 14%|█▍        | 29.2M/211M [00:00<00:01, 97.6MiB/s]
 19%|█▊        | 39.2M/211M [00:00<00:01, 98.5MiB/s]
 23%|██▎       | 49.0M/211M [00:00<00:01, 97.4MiB/s]
 28%|██▊       | 58.8M/211M [00:00<00:01, 93.6MiB/s]
 33%|███▎      | 69.3M/211M [00:00<00:01, 97.1MiB/s]
 38%|███▊      | 79.0M/211M [00:00<00:01, 96.0MiB/s]
 42%|████▏     | 89.4M/211M [00:00<00:01, 98.5MiB/s]
 48%|████▊     | 100M/211M [00:01<00:01, 101MiB/s]  
 52%|█████▏    | 110M/211M [00:01<00:01, 99.5MiB/s]
 57%|█████▋    | 120M/211M [00:01<00:00, 99.1MiB/s]
 62%|██████▏   | 131M/211M [00:01<00:00, 102MiB/s] 
 67%|██████▋   | 141M/211M [00:01<00:00, 101MiB/s]
 72%|███████▏  | 151M/211M [00:01<00:00, 99.9MiB/s]
 77%|███████▋  | 162M/211M [00:01<00:00, 102MiB/s] 
 82%|████████▏ | 173M/211M [00:01<00:00, 103MiB/s]
 87%|███

 > Downloading XTTS v2.0 files!


100%|██████████| 211M/211M [00:02<00:00, 93.9MiB/s]

361kiB [00:00, 1.78MiB/s]7G [00:00<?, ?iB/s]

  1%|          | 9.80M/1.87G [00:00<00:18, 98.0MiB/s]
  1%|          | 20.7M/1.87G [00:00<00:17, 105MiB/s] 
  2%|▏         | 31.2M/1.87G [00:00<00:18, 100MiB/s]
  2%|▏         | 41.6M/1.87G [00:00<00:17, 102MiB/s]
  3%|▎         | 53.1M/1.87G [00:00<00:17, 107MiB/s]
  3%|▎         | 64.6M/1.87G [00:00<00:16, 109MiB/s]
  4%|▍         | 75.7M/1.87G [00:00<00:16, 110MiB/s]
  5%|▍         | 86.8M/1.87G [00:00<00:16, 110MiB/s]
  5%|▌         | 98.1M/1.87G [00:00<00:15, 111MiB/s]
  6%|▌         | 109M/1.87G [00:01<00:15, 110MiB/s] 
  6%|▋         | 121M/1.87G [00:01<00:15, 111MiB/s]
  7%|▋         | 132M/1.87G [00:01<00:15, 112MiB/s]
  8%|▊         | 143M/1.87G [00:01<00:15, 109MiB/s]
  8%|▊         | 154M/1.87G [00:01<00:17, 97.0MiB/s]
  9%|▉         | 164M/1.87G [00:01<00:17, 96.1MiB/s]
  9%|▉         | 175M/1.87G [00:01<00:17, 98.8MiB/s]
 10%|▉         | 186M/1.87G [00:01<00:16, 104MiB/s] 
 

In [10]:
training_dir = "/kaggle/input/ljspeech-gothic/15"

In [11]:
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  
START_WITH_EVAL = True
BATCH_SIZE = 1
GRAD_ACUMM_STEPS = 252
LANGUAGE = "ru"

In [12]:
model_args = GPTArgs(
    max_conditioning_length=143677,#the audio you will use for conditioning latents should be less than this 
    min_conditioning_length=9185,#and more than this
    debug_loading_failures=True,#this will print output to console and help you find problems in your ds
    max_wav_length=216346,#set this to >= the longest audio in your dataset  
    max_text_length=200, 
    mel_norm_file=MEL_NORM_FILE,
    dvae_checkpoint=DVAE_CHECKPOINT,
    xtts_checkpoint=XTTS_CHECKPOINT,  
    tokenizer_file=TOKENIZER_FILE,
    gpt_num_audio_tokens=1026, 
    gpt_start_audio_token=1024,
    gpt_stop_audio_token=1025,
    gpt_use_masking_gt_prompt_approach=True,
    gpt_use_perceiver_resampler=True,
)

In [13]:
audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000) 

In [ ]:
reference_candidates = [
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_KHARIM_CHARGE_INSULT_STUPID_15_00.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_BLOODWYN_FORGETIT_15_00.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/TPL_1401_GORNAKOSH_SUGGEST_INFO_15_01.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/DIA_DUSTY_METMELVIN_15_02.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/DIA_DEXTER_JOINOC_ADVANCE_15_00.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_DIEGO_OCSTORY_15_04.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_SATURAS_YBERION_15_04.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_XARDASDEMON_MASTERWHO_15_01.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/DIA_CRONOS_BANDIT_15_00.wav",
]

In [ ]:
bad_samples = [
    "/kaggle/input/ljspeech-gothic/15/wavs/DIA_GRD_218_FIRSTIN_15_01.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_KIRGO_CHARGE_BEER_15_00.wav",
]

In [ ]:
[
    sample
    for sample in train_samples
    if sample["audio_file"] in bad_samples
]

In [15]:
array, sr = librosa.load("/kaggle/input/ljspeech-gothic/15/wavs/DIA_DUSTY_METMELVIN_15_02.wav")
Audio(array, rate=sr)

In [ ]:
for sample in (train_samples + eval_samples)[100:200]:
    print("text:\t%spath:\t%s" % (sample["text"], sample["audio_file"]))
    array, sr = librosa.load(sample["audio_file"])
    display(Audio(array, rate=sr))

In [ ]:
train_samples[0]

In [16]:
SPEAKER_REFERENCE = "/kaggle/input/ljspeech-gothic/15/wavs/DIA_DUSTY_METMELVIN_15_02.wav"

In [17]:
config = GPTTrainerConfig(
    run_eval=True,
    epochs = 1000, # assuming you want to end training manually w/ keyboard interrupt
    output_path=OUT_PATH,
    model_args=model_args,
    run_name="xtts-gothic-ich",
    project_name="1",
    run_description="""
        GPT XTTS training
        """,
    # dashboard_logger=DASHBOARD_LOGGER,
    wandb_entity=None,
    # logger_uri=LOGGER_URI,
    audio=audio_config,
    batch_size=BATCH_SIZE,
    batch_group_size=48,
    eval_batch_size=BATCH_SIZE,
    num_loader_workers=8, #consider decreasing if your jupyter env is crashing or similar
    eval_split_max_size=256, 
    print_step=50, 
    plot_step=100, 
    log_model_step=1000, 
    save_step=9999999999, #ALREADY SAVES EVERY EPOCHMaking this high on kaggle because Output dir is limited in size. I changed this to be size of training set/2 so I would effectively have a checkpoint every half epoch 
    save_n_checkpoints=1,#if you want to store multiple checkpoint rather than just 1, increase this
    save_checkpoints=False,# Making this False on kaggle because Output dir is limited
    print_eval=False,
    optimizer="AdamW",
    optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
    optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
    lr=5e-06,  
    lr_scheduler="MultiStepLR",
    lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
    test_sentences=[ 
        {
            "text": "Скорее всего его убили стражники из Старого лагеря.",
            "speaker_wav": SPEAKER_REFERENCE, 
            "language": LANGUAGE,
        },
        {
            "text": "Магам воды и наемникам не нравилось, что мы совершаем набеги на конвои из Старого лагеря.",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
        {
            "text": "Эй, парни, идите сюда! У нас предатель!",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
        {
            "text": "Привет, человек",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
        {
            "text": "Ну, сколько можно тебя ждать?",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
        {
            "text": "Ты сам напросился!",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
        {
            "text": "Это лагерь бандитов, парень.",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
        {
            "text": "Франко наш новый руководитель.",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
    ],
) 

model = GPTTrainer.init_from_config(config)

In [ ]:
experiment_config = comet_ml.ExperimentConfig(
    name="hero-2"
)

exp = comet_ml.start(experiment_config=experiment_config)

In [ ]:
def my_callback(trainer):
    audios = trainer.model.test_run({})["audios"]
    for name, audio_array in audios.items():
        display(Audio(audio_array, rate=24000))
        # exp.log_metrics(trainer.best_loss, step=trainer.epochs_done)
        exp.log_audio(
            audio_data=audio_array,
            sample_rate=24000,
            file_name=name,
            step=trainer.epochs_done,
        )
        

In [ ]:
trainer = Trainer(
    TrainerArgs(
        restore_path="/kaggle/input/xtts-hero-gothic/pytorch/default/2/xtts-hero-model/best_model.pth",
        # restore_path=None,
        skip_train_epoch=False,
        start_with_eval=START_WITH_EVAL,
        grad_accum_steps=GRAD_ACUMM_STEPS,
    ),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    callbacks={"on_epoch_end": my_callback},
)
trainer.fit()

In [ ]:
!rm -rf /kaggle/working/run/xtts-gothic-ich-September-04-2025_07+07AM-0000000

In [ ]:
# XTTS_CHECKPOINT = os.path.join(RUN_DIR, 'best_model.pth')
# CONFIG_PATH = os.path.join(RUN_DIR, 'config.json')
# TOKENIZER_PATH = os.path.join(OUT_PATH, 'XTTS_v2.0_original_model_files', 'vocab.json')
# SPEAKER_REFERENCE = os.path.join(notebook_dir, 'training_data', 'goreNewChunking', 'wavs', 'indepth1_SPEAKER_04_segment_00234.wav') #booksbreakfast_SPEAKER_02_segment_00145.wav begins to alter

# SPEAKER_REFERENCE = os.path.join(notebook_dir, 'training_data', 'goreOutliersRemovedToplossesculled', 'wavs', '80sspeeches_SPEAKER_00_segment_00027.wav')#one second 'i went to exeter too' 80sspeeches_SPEAKER_00_segment_00014
# OUTPUT_WAV_PATH = os.path.join(notebook_dir, 'outs')

# print("Loading model…")
# config = XttsConfig()
# config.load_json(CONFIG_PATH)
# model = Xtts.init_from_config(config)
# model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
# model.cuda()

# print("Computing speaker latents…")
# gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])

# def sample(prompt, name, temperature=0.7, speed=1.0):
#     out = model.inference(
#         text=prompt,
#         language="en",
#         gpt_cond_latent=gpt_cond_latent,
#         speaker_embedding=speaker_embedding,
#         temperature=temperature, #TODO: what other optional args are there
#         speed=speed,
#         enable_text_splitting=True
#     )
#     output_path = os.path.join(OUTPUT_WAV_PATH, f'{name}{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.wav')
#     torchaudio.save(output_path, torch.tensor(out["wav"]).unsqueeze(0), 24000)
    
#     sample("this is some speech that will be generated", 'longtest' + 'defaultboundarymid'+ cond.split('.')[0], temperature = 0.3)



In [ ]:
from TTS.tts.models.xtts import Xtts
from TTS.tts.configs.xtts_config import XttsConfig

In [ ]:
CONFIG_PATH = "/kaggle/working/run/xtts-gothic-ich-September-03-2025_11+33AM-0000000/config.json"
XTTS_CHECKPOINT = "/kaggle/working/run/xtts-gothic-ich-September-03-2025_11+33AM-0000000/best_model.pth"
TOKENIZER_PATH = "/kaggle/working/run/XTTS_v2.0_original_model_files/vocab.json"

In [ ]:
config = XttsConfig()
config.load_json(CONFIG_PATH)
new_model = Xtts.init_from_config(config)
new_model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
new_model.cuda()

In [ ]:
gpt_cond_latent, speaker_embedding = new_model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])
out = new_model.inference(
    text="Ну, сколько можно тебя ждать?",
    language="ru",
    gpt_cond_latent=gpt_cond_latent,
    speaker_embedding=speaker_embedding,
    temperature=0.6, #TODO: what other optional args are there
    speed=1.5,
    enable_text_splitting=True
)
Audio(out["wav"], rate=24000)

In [ ]:
prompts = [
    "Привет, человек",
    # Пачо
    # "Info_Pacho_GRD_FM_13_01"
    "Ну, сколько можно тебя ждать?",
    # "Info_Pacho_GRD_FM_13_03"
    "Ладно, все равно он сам сильно задерживается.",
    # # "Info_Pacho_GRD_FM_END_13_01"
    "А куда это ты собрался?",
    # # "Info_Pacho_GRD_FM_END_13_03"
    "Эй, парни, идите сюда! У нас предатель!",

    # "Info_Org_867_Raeuber_BDT_13_00"
    "Я получаю достаточно, как стражник лагеря.",
    # "Org_867_Raeuber_About_13_00"
    "Квентин убит.",
    # "Org_867_Raeuber_About_13_01"
    "Скорее всего его убили стражники из Старого лагеря.",
    # "Org_867_Raeuber_About_13_02"
    "Франко наш новый руководитель.",
    # "Org_867_Raeuber_About_13_03"
    "Все, как обычно.",
    # "Org_867_Raeuber_Banditencamp_13_01"
    "Это лагерь бандитов, парень.",
    # "Org_867_Raeuber_GUARD_Info_13_00"
    "Стой!",
    # "Org_867_Raeuber_GUARD_Info_13_02"
    "Тебе здесь нечего делать, парень.",
    # "Org_867_Raeuber_GoAway_Info_13_00"
    "Еще один придурок из Старого лагеря.",
    # "Org_867_Raeuber_Main_13_00"
    "Сейчас нас возглавляет Франко.",
    # "Org_867_Raeuber_Main_13_01"
    "Квентин главный в лагере.",
    # "Org_867_Raeuber_Organisators_13_01"
    "Больше нет.",
    # "Org_867_Raeuber_Organisators_13_02"
    "Магам воды и наемникам не нравилось, что мы совершаем набеги на конвои из Старого лагеря.",
    # "Org_867_Raeuber_Organisators_13_03"
    "Ларес предложил части воров уйти из Нового лагеря и посроить свой лагерь здесь.",
    # "Org_867_Raeuber_Organisators_13_04"
    "Недавно мы напали еще на один отряд стражников.",
    # "Org_867_Raeuber_Organisators_13_05"
    "В лагере. Пройди дальше через ущелье.",
    # "Org_867_Raeuber_Organisators_13_06"
    "Еще трое крутят болотник в пещере около Нового лагеря.",
    # "Org_867_Raeuber_WereIsMain_13_00"
    "Он в лагере. Просто иди дальше через ущелье.",
    # "Org_867_Raeuber_WereIsMain_13_01"
    "Он куда-то ушел из лагеря пару дней назад.",

    # Франко
    # "Info_Org_869_Raeuber_BDT_13_00"
    "А ты что думал? Он же глава банды.",
    # "Info_Org_869_Raeuber_TRADE_13_00"
    "Да, у меня есть куча товара с нашего склада.",

    # "SVM_13_YouAskedForIt"
    "Ты сам напросился!",
]


In [ ]:
tts = TTS("/kaggle/working/run/xtts-gothic-ich-September-03-2025_11+33AM-0000000").to('cuda')

In [ ]:
reference_candidates = [
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_KHARIM_CHARGE_INSULT_STUPID_15_00.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_BLOODWYN_FORGETIT_15_00.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/TPL_1401_GORNAKOSH_SUGGEST_INFO_15_01.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/DIA_DUSTY_METMELVIN_15_02.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/DIA_DEXTER_JOINOC_ADVANCE_15_00.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_DIEGO_OCSTORY_15_04.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_SATURAS_YBERION_15_04.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/INFO_XARDASDEMON_MASTERWHO_15_01.wav",
    "/kaggle/input/ljspeech-gothic/15/wavs/DIA_CRONOS_BANDIT_15_00.wav",
]

In [ ]:
wav = tts.tts(
    text="Привет, человек!",
    speaker_wav=SPEAKER_REFERENCE,
    language="ru",
    speed=1.0,
    temperature=0.5
)
Audio(wav, rate=24000)

In [ ]:
Audio(wav, rate=24000)

In [ ]:
array, sr = librosa.load("/kaggle/input/ljspeech-gothic/15/wavs/DIA_DUSTY_METMELVIN_15_02.wav")
Audio(array, rate=sr)

In [ ]:
wav = tts.tts(
    # text="Ну, сколько можно тебя ждать?",
    text="Франко наш новый руководитель.",
    speaker_wav="/kaggle/input/ljspeech-gothic/15/wavs/DIA_DUSTY_METMELVIN_15_02.wav",
    language="ru"
)
Audio(wav, rate=24000)

In [ ]:
for prompt in prompts:
    print(prompt)
    for _ in range(3):
        display(
            Audio(
                tts.tts(text=prompt, speaker_wav=SPEAKER_REFERENCE, language="ru"),
                rate=24000
            ),
        )

In [ ]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to('cuda')

In [ ]:
for prompt in prompts:
    print(prompt)
    for _ in range(3):
        display(
            Audio(
                tts.tts(text=prompt, speaker_wav=SPEAKER_REFERENCE, language="ru"),
                rate=24000
            ),
        )

In [ ]:
wav = tts.tts(
    text="Привет, человек!",
    speaker_wav="/kaggle/input/audioaa/DIA_DARRION_HELLO_15_00.WAV",
    language="ru"
)

In [ ]:
Audio(wav, rate=24000)

In [ ]:
model.tts_to_file(
    text='Привет, человек!',
    speaker_wav="/kaggle/input/audioaa/DIA_DARRION_HELLO_15_00.WAV",
    language="ru",
    file_path=audio_file_name
)